# Importing the necessary packages

In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import nltk
import math
nltk.download('stopwords')
from nltk.corpus import stopwords
from sklearn import svm

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\ACER\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


# Importing the Spam Mails dataset from Kaggle

In [4]:
df =   pd.read_csv('https://raw.githubusercontent.com/GaneshDev2003/PRML-Assignment-3/main/spam_ham_dataset.csv')
df.columns = ['index', 'label', 'text', 'label_num']
X = df['text'].values
y = df['label_num'].values
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split( X, y, test_size=0.2, random_state=0)
print(len(X_test))

1035


# Preprocessing
The email text is preprocessed using the following steps:


1.  Punctuation removal
2.  Stopword removal
3.  Feature vector extraction




In [5]:
def remove_punctutations(word):
  list_of_punctutations = [",", ".", "?", "!", "'", "+", "(", ")", "{", "}", ";" , "\\", "/", ":", "-", "=", "_", "@", "$", "%", "^", "&", "*", "#"]
  new_word = ""
  for character in word:
    if(character not in list_of_punctutations):
      new_word = new_word + character
  return new_word

In [6]:
def remove_stopwords(list_of_words):
  stop_words = set(stopwords.words('english'))
  filtered_words = []
  for word in list_of_words:
    if(word.lower() not in stop_words):
      filtered_words.append(word)
  return filtered_words

In [7]:
def remove_digits(word):
  return(''.join([i for i in word if not i.isdigit()]))

In [8]:
def preprocess(email):
  tokens = []
  for word in email.split():
    word = remove_punctutations(word)
    word = remove_digits(word)
    tokens.append(word.lower())
  tokens = remove_stopwords(tokens)
  tokens = [token for token in tokens if token]
  tokens = set(tokens)
  
  return list(tokens)

In [9]:
dictionary = []
tokens_list = []
for email in X_train:
  tokens = preprocess(email)
  tokens_list.append(tokens)
  dictionary = dictionary + tokens
dictionary = list(set(dictionary))
print(len(dictionary))

39133


In [10]:
def extract_feature_vector(email_tokens, unique_words):
  temp = np.arange(len(unique_words))
  j=0
  for word in unique_words:
    temp[j] = int(word in email_tokens)
    j = j+1
  return temp

In [ ]:
feature_vec = np.zeros((len(tokens_list), len(dictionary)))
i=0
for email_tokens in tokens_list:
  feature_vec[i] = extract_feature_vector(email_tokens, dictionary)
  i = i+1
feature_vec = np.array(feature_vec, dtype = object)

# Naive Bayes Classifier

In [49]:
train_spam_indices = [i for i in range(len(X_train)) if y_train[i] == 1]
train_ham_indices = [i for i in range(len(X_train)) if y_train[i] == 0]
train_spam_tokens = [tokens_list[i] for i in train_spam_indices]
train_ham_tokens = [tokens_list[i] for i in train_ham_indices]

In [50]:
prob_spam_ham = len(train_spam_indices)/(len(train_spam_indices)+ len(train_ham_indices))
print(len(train_spam_indices))
print(len(train_ham_indices))
print(prob_spam_ham)

1196
2940
0.28916827852998067


In [51]:
def predict(email):
  test_email_tokens = preprocess(email)
  n = 0
  prob_spam = math.log(prob_spam_ham)
  prob_ham = math.log(1-prob_spam_ham)
  for token in test_email_tokens:
    #no. of spam emails containing word
    count_spam = 0
    #no. of ham emails containing word
    count_ham = 0
    for spam_tokens in train_spam_tokens:
      if token in spam_tokens:
        count_spam = count_spam + 1
    for ham_tokens in train_ham_tokens:
      if token in ham_tokens:
        count_ham = count_ham + 1
    if(count_spam == 0 and count_ham == 0):
      pass
    prob_spam += math.log((count_spam + 1) / (len(train_spam_indices) + 2))
    prob_ham += math.log((count_ham + 1) / (len(train_ham_indices) + 2))
  if(prob_spam > prob_ham):
    return 1
  else:
    return 0

In [52]:
i = 15
print(str(predict(X_test[i])) + " " + str(y_test[i]))

0 0


In [ ]:
count = 0
for i in range(len(X_test)):
  email = X_test[i]
  if(predict(email) == y_test[i]):
    count = count+1
    #print(df_test.iloc[i,1])
print((count/len(y_test))*100)

93.33333333333333


# Logistic Regression

In [41]:
email_train = []
for email in X_train:
  email_train.append(extract_feature_vector(email, dictionary))

In [42]:
def sigmoid(x,w):
  return (1 / (1 + np.exp(-1*np.dot(x,w))))

In [43]:
def gradient(X, y, w, learning_rate):
  grad = 0
  i = 0
  for i in range(len(X)):
    grad = grad + X[i]*(y[i] - sigmoid(X[i],w))
    i = i + 1
  return grad

In [44]:
def gradient_descent(X,y, learning_rate, number_of_epochs):
  w = [0]*X.shape[1]
  w = np.array(w)
  print(w.shape)
  for i in range(number_of_epochs):
    w = w + learning_rate*gradient(X,y,w,learning_rate)
  return w

In [45]:
X_train_features = np.array(email_train)
w_gd = gradient_descent(X_train_features, y_train, 0.01, 200)

(39133,)


C:\Users\ACER\AppData\Local\Temp\ipykernel_10600\95869451.py:2: RuntimeWarning: overflow encountered in exp
  return (1 / (1 + np.exp(-1*np.dot(x,w))))


In [46]:
def predict_logistic(email):
  email_features = extract_feature_vector(email, dictionary)
  if(sigmoid(email_features, w_gd)>0.5):
    return 1
  else:
    return 0

In [47]:
i = 100
print(str(predict_logistic(X_test[i])) + " " + str(y_test[i]))

0 0


In [48]:
count = 0  
for i in range(len(y_test)):
  if(predict_logistic(X_test[i]) == y_test[i]):
    count = count+1
print(count/len(y_test))

C:\Users\ACER\AppData\Local\Temp\ipykernel_10600\95869451.py:2: RuntimeWarning: overflow encountered in exp
  return (1 / (1 + np.exp(-1*np.dot(x,w))))


0.9642512077294686


# Testing the algorithm using emails in the test folder

In [34]:
import os
def test_emails_from_folder():    
    path = r"C:\Users\ACER\Downloads\PRMLAssignment3\test"
    os.chdir(path)
    os.listdir()
    for file in os.listdir():
        if(file.endswith('.txt')):
            file_path = f"{path}\{file}"
            with open(file_path, 'r') as f:
                print(predict_logistic(f.read()))

test_emails_from_folder()

0
1


In [64]:
print(X_test[3])

Subject: via - ggra is lousy mizar
anti
curb
hemisphere
% q
news ; blogs ; white papers ; downloads ; reviews ; prices . go . alerts | ; newsletters | ;
rss feeds . click here . enterprise news . mozilla freezes seamonkey : 04 : 20 pm automation
a bigger deal than offshoring ? 03 : 33 pm . time to kill it for the



In [37]:
print(df.shape)

(5171, 4)
